# Calibrating 2016 ANES data by linear regression.

In [1]:
options(jupyter.rich_display=FALSE) # Create output as usual in R

The following makes use of the *survey* package. You may need to install it from
[CRAN](https://cran.r-project.org/package=survey) using the code
`install.packages("survey")` if you want to run this on your computer. (The
package is already installed on the notebook container, however.)

In [2]:
library(survey)

Loading required package: grid

Loading required package: Matrix

Loading required package: survival


Attaching package: ‘survey’


The following object is masked from ‘package:graphics’:

    dotchart




The following loads data created in an earlier example.

In [3]:
load("anes-2016-vprevote-design.RData")

`calibrate` expects the names of the calibration vectors to be like those of
regression coefficents.

In [4]:
cal_names(~recall12+vote16,anes_2016_vprevote_desgn)

[1] "(Intercept)"     "recall12Romney"  "recall12Other"   "recall12No vote"
[5] "vote16Trump"     "vote16Other"     "vote16No vote"  

The following code defines a function creates a vector suitable for calibration from the
data frames that `postStratify()` or `rake()` use

In [5]:
calib_counts <- function(formula,frames){
    dframe2coef <- function(data){
        fname <- names(data)[1]
        flevels <- as.character(data[[1]])
        Freq <- data$Freq
        coefs <- c(sum(Freq),Freq[-1])
        names(coefs) <- c("(Intercept)",
                          paste0(fname,flevels[-1]))
        coefs
    }
    vars <- all.vars(formula)
    for(i in seq_along(vars)){
        var_i <- vars[i]
        frame_i <- frames[[var_i]]
        coef_i <- dframe2coef(frame_i)
        if(i==1)
            res <- coef_i
        else
            res <- c(res,coef_i[-1])
    }
    res
}

We now apply this function to get the calibration criteria. The file "popl-results.RData" contains the population-level ata.

In [6]:
load("popl-results.RData")
calib_anes16 <- calib_counts(~recall12+vote16,
                             list(recall12=pop.recall12,
                                  vote16=pop.vote16))

Finally we calibrate the ANES sample.

In [7]:
anes_2016_prevote_desgn_calib <- calibrate(
    anes_2016_vprevote_desgn,~recall12+vote16,
    population=calib_anes16)

In [8]:
100*svymean(~recall12,design=anes_2016_prevote_desgn_calib)

                    mean SE
recall12Obama   28.01970  0
recall12Romney  25.90182  0
recall12Other    0.95053  0
recall12No vote 45.12795  0

In [9]:
100*svymean(~vote16,design=anes_2016_prevote_desgn_calib)

                 mean SE
vote16Clinton 26.3355  0
vote16Trump   25.1883  0
vote16Other    3.1324  0
vote16No vote 45.3439  0

In [10]:
save(anes_2016_prevote_desgn_calib,file="anes-2016-prevote-desgn-calib.RData")